<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Tomás Fontecilla </em><br>

</div>

# Machine Learning Avanzado
*17 de Diciembre de 2024*

#### Integrantes: 
`Jocelyn Cáceres, Kurt Castro, Giuseppe Lavarello, Carlos Saquel`

1) LSTM producción de leche
    - Preparación de Datos
    - Visualización serie de tiempo
    - Análisis serie de tiempo? tendencia, estacionalidad, etc
    - Modelo LSTM
        - Separar datos (train, test, ¿val?)
        - Diseño arquitectura modelo
        - Entrenar modelo
        - Validacion y calculo de metricas
        - predicciones (pide 3 meses)
            - Visualizar predicciones
            - Comparar resultados

2) LSTM producción IPSA
    - Obtencion de datos (probablemente directo de la pagina de la bolsa de santiago)
    - Preparación de Datos
    - Visualización serie de tiempo
    - Análisis serie de tiempo?
    - Modelo LSTM
        - Separar datos (train, test, ¿val?)
        - Diseño arquitectura modelo
        - Entrenar modelo
        - Validacion y calculo de metricas
        - predicciones (pide 3 meses)
            - Visualizar predicciones
            - Comparar resultados
3) Autoencoder, limpieza de imágenes y extracción de texto
    - Preparación de Datos
    - Visualización de muestra de datos
    - Preprosesamiento de imagenes
    - Autoencoder
        - Separar imagenes (train, test, val)
        - Diseño arquitectura modelo
        - Entrenar modelo
        - Limpiar imagenes y visualizar comparacion de 2 o 3 ejemplos
        - Extraer texto y validar 
        - Usar imagen propia "mostrar imagen inicial y después de correr el modelo"

4) Conclusiones y dar formato de Informe
    - Ejemplo: 
        1. Objetivo
        2. Introducción
        3. Metodología 
            - Aca el codigo y las validaciones
        4. Conclusiones    
